In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import speed_of_light, Boltzmann, Planck
from functools import reduce
from scipy.integrate import solve_ivp

In [2]:
from ROSAA_func import distribution, boltzman_distribution, \
    stimulated_absorption, stimulated_emission,\
    voigt, lorrentz_fwhm, gauss_fwhm

In [3]:
import pprint
pp = pprint.PrettyPrinter()
pprint = pp.pprint

In [4]:
def lineshape_normalise():

    freq = float(main_parameters["freq"])  # transition frequency in Hz

    # doppler line width

    massIon = float(lineshape_conditions["IonMass(amu)"])
    tempIon = float(lineshape_conditions["IonTemperature(K)"])
    sigma = gauss_fwhm(freq, massIon, tempIon)

    # power broadening
    dipoleMoment = float(power_broadening["dipoleMoment(D)"])
    power = float(power_broadening["power(W)"])

    cp = float(power_broadening["cp"])
    gamma = lorrentz_fwhm(dipoleMoment, power, cp)

    # normalised line shape factor
    LineShape = voigt(gamma, sigma)
    
    # transition rate due to influence of mm-wave 
    # normalisation factor

    trap_area = float(main_parameters["trap_area"])
    norm = (power/(trap_area*speed_of_light))*LineShape
    
    print(f"{massIon=}\n{tempIon=}\n{sigma=:.2e}\n{gamma=:.2e}\n{LineShape=:.2e}\n{norm=:.2e}\n")
    return norm


def getCollisionalRate(collisional_rates):
    
    rates = {}
    
    for i in range(totallevel):
        for j in range(totallevel):
            if i != j & j>i:
                deexciteRateConstantKey = f"q_{j}{i}"
                exciteRateConstantKey = f"q_{i}{j}"
                
                if q_deexcitation_mode:
                    
                    _temp = collisional_rates[deexciteRateConstantKey]
                    rates[deexciteRateConstantKey] = _temp
                    rates[exciteRateConstantKey] = _temp * distribution(i, j, Energy, trapTemp)
                    
                else:
                    _temp = collisional_rates[exciteRateConstantKey]
                    rates[exciteRateConstantKey] = _temp
                    rates[deexciteRateConstantKey] = _temp * distribution(j, i, Energy, trapTemp)
    return rates

def getAttachmentRates():
    
    Rate_k31_0 = k31_0*nHe**2
    Rate_k31_1 = k31_1*nHe**2
    Rate_k32 = k32*nHe**2
    Rate_kCID1 = kCID1*nHe
    Rate_kCID2 = kCID2*nHe
    
    return Rate_k31_0, Rate_k31_1, Rate_k32, Rate_kCID1, Rate_kCID2

NameError: name 'totallevel' is not defined

In [49]:
def computeAttachmentProcess(N, N_He, dR_dt):
    
    # attachmentRate0, attachmentRate1, attachmentRate2 = get_attachment_rates(N, N_He)
    # Adding rare gas atom attachment and dissociation rates

    attachmentRate0 = - Rate_k31_0*N[0] + Rate_kCID1*N_He[0]*p
    attachmentRate1 = - Rate_k31_1*N[1] + Rate_kCID1*N_He[0]*(1-p)
    attachmentRate2 = - Rate_k32*N_He[0] + Rate_kCID2*N_He[1]

    dR_dt[0] += attachmentRate0
    dR_dt[1] += attachmentRate1

    # CDHe:
    dCDHe_dt = - attachmentRate0 - attachmentRate1 + attachmentRate2
    dR_dt.append(dCDHe_dt)

    # CDHe2
    dCDHe2_dt = - attachmentRate2
    dR_dt.append(dCDHe2_dt)

    return dR_dt

def computeCollisionalProcess(i, N):
    
    collections = []

    for j in range(totallevel):
        if i!= j: 
            
            key = f"q_{j}{i}"
            keyInverse = f"q_{i}{j}"
            
            k = collisional_rates[key]*nHe*N[j] - collisional_rates[keyInverse]*nHe*N[i]
            collections.append(k)
    
    if testMode: print(f"collisional_collection: {i=}\t{collections=}")
    return collections

def computeEinsteinProcess(i, N):
    
    collections = []
    
    if includeSpontaneousEmission:

        # Einstein Coefficient A
        if i == 0: 
            temp = A_10*N[excitedTo]
            collections.append(temp)

        if i == 1:
            temp = -A_10*N[excitedTo]
            collections.append(temp)

    # Einstein Coefficient B

    if lightON:
        
        # B_rate defined from excited state 
        B_rate = B_01*N[excitedFrom] - B_10*N[excitedTo]

        if i==excitedFrom:
            temp = -B_rate
            collections.append(temp)

        if i==excitedTo:
            temp = B_rate
            collections.append(temp)
        
        if testMode: print(f"{excitedTo=}\t{B_rate=}\t{-B_rate=}")
        
    if testMode: print(f"einstein_collection: {i=}\t{collections=}")
    return collections

In [50]:
def computeRateDistributionEquations(t, counts):
    
    if testMode: print(f"{counts=}\n")
    
    if includeAttachmentRate:
        N =  counts[:-2]
        N_He = [counts[-2], counts[-1]]
    else:
        N = counts
    
    rateCollection = []
    
    for i in range(totallevel):
        
        collisional_collection = computeCollisionalProcess(i, N)
        einstein_collection = computeEinsteinProcess(i, N)
        
        collections = collisional_collection + einstein_collection
        rateCollection.append(collections)
        if testMode: print(f"{rateCollection=}")
        
    dR_dt = []
    for _ in rateCollection:
        _temp = reduce(lambda a, b: a+b, _)
        dR_dt.append(_temp)
        
    if includeAttachmentRate:
        dR_dt = computeAttachmentProcess(N, N_He, dR_dt)
    
    return dR_dt

In [140]:
conditions={'trapTemp': 5.7, 'variable': 'time', 'variableRange': '1e12,  1e16,  10', 'includeCollision': True, 'writefile': True, 'filename': 'ROSAA_modal_CD_He', 
            'currentLocation': 'Z:\\Students\\Aravindh\\Measurements\\CO+', 'q_deexcitation_mode': True, 
            'collisional_rates': {'q_10':4.129378677912417e-11, 'q_20': 3.9250811501906335e-11, 'q_21': 1.2457658629399757e-10, \
                                  'q_30': 1.3300641305139243e-11, 'q_31': 8.492073041429557e-11, 'q_32': 6.81110895320521e-11}, 
            'main_parameters': {'molecule': 'CD', 'tagging partner': 'He', 'freq': '453_521_850_000', 'trap_area': '5e-5', 'Energy': '0, 15.127861, 45.373851, 90.718526, 151.132755, 226.577764'}, 
            'simulation_parameters': {'totalIonCounts': 1000, 'Simulation time(ms)': 600, 'Total steps': 1000, 'numberOfLevel (J levels)': 4, 'excitedTo':1, 'excitedFrom':0}, 
            'einstein_coefficient': {'A_10':'6.24e-4', 'spontaneous_emissions': [],  "includeSpontaneousEmission":True}, 
            'power_broadening': {'cp': '4.9e7', 'dipoleMoment(D)': 0, 'power(W)': '2e-5'}, 
            'lineshape_conditions': {'IonMass(amu)': 14, 'IonTemperature(K)': 12.3}, 
            'rate_coefficients': {'branching-ratio': 0.5, 'a': 0.5, 'includeAttachmentRate':False, 'He density(cm3)': '5e14', 'k3': '9.6e-31,  2.9e-30', 'kCID': '6.7e-16,  1.9e-15'}
        }

main_parameters = conditions["main_parameters"]
simulation_parameters = conditions["simulation_parameters"]
einstein_coefficient = conditions["einstein_coefficient"]
lineshape_conditions = conditions["lineshape_conditions"]
rate_coefficients = conditions["rate_coefficients"]
power_broadening = conditions["power_broadening"]

In [141]:
nHe = float(rate_coefficients["He density(cm3)"])
print(f"{nHe=:.2e}\n")

k31_0, k32 = [float(i.strip()) for i in rate_coefficients["k3"].split(",")]
a = float(rate_coefficients["a"])
k31_1 = a*k31_0

kCID1, kCID2 = [float(i.strip()) for i in rate_coefficients["kCID"].split(",")]
Rate_k31_0, Rate_k31_1, Rate_k32, Rate_kCID1, Rate_kCID2 =  getAttachmentRates()

print(f"k31_1 dependent factor: {a}\n")
print(f"{k31_0=:.2e}\t{Rate_k31_0=:.2f}")
print(f"{k31_1=:.2e}\t{Rate_k31_1=:.2f}")
print(f"{k32=:.2e}\t{Rate_k32=:.2f}\n")
print(f"{kCID1=:.2e}\t{Rate_kCID1=:.2f}")
print(f"{kCID2=:.2e}\t{Rate_kCID2=:.2f}\n")

p = float(rate_coefficients["branching-ratio"])
print(f"Branching Ratio: {p}\n")

nHe=5.00e+14

k31_1 dependent factor: 0.5

k31_0=9.60e-31	Rate_k31_0=0.24
k31_1=4.80e-31	Rate_k31_1=0.12
k32=2.90e-30	Rate_k32=0.72

kCID1=6.70e-16	Rate_kCID1=0.34
kCID2=1.90e-15	Rate_kCID2=0.95

Branching Ratio: 0.5



In [142]:
includeSpontaneousEmission = einstein_coefficient["includeSpontaneousEmission"]
includeCollision = conditions["includeCollision"]
includeAttachmentRate = rate_coefficients["includeAttachmentRate"]
print(f"{includeAttachmentRate=}\n{includeCollision=}\n{includeSpontaneousEmission=}\n")

totallevel = simulation_parameters["numberOfLevel (J levels)"]
print(f"{totallevel=}\n")

Energy = [float(_) for _ in main_parameters["Energy"].split(", ")][:totallevel]
trapTemp = conditions["trapTemp"]
print(f"{Energy=} in cm-1\n{trapTemp=}K")

excitedTo = simulation_parameters["excitedTo"]
excitedFrom = simulation_parameters["excitedFrom"]

freq = float(main_parameters["freq"])  # transition frequency in Hz
print(f"{freq=:.2e} Hz\n")

norm = lineshape_normalise()

A_10 = float(einstein_coefficient["A_10"])
print(f"{A_10=:.2e}\n")

B_10 = stimulated_emission(A_10, freq)*norm
B_01 = stimulated_absorption(excitedFrom, excitedTo, B_10)
print(f"{B_10=:.2e}\t{B_01=:.2e}\n")

collisional_rates = {q:float(value) for q, value in conditions["collisional_rates"].items()}
q_deexcitation_mode = conditions["q_deexcitation_mode"]
collisional_rates = getCollisionalRate(collisional_rates)

print(f"Collisional Rates")
for key, value in collisional_rates.items():
    print(f"{key}: {value:.2e}\t{key}*nHe: {value*nHe:.2e}\n")

includeAttachmentRate=False
includeCollision=True
includeSpontaneousEmission=True

totallevel=4

Energy=[0.0, 15.127861, 45.373851, 90.718526] in cm-1
trapTemp=5.7K
freq=4.54e+11 Hz

massIon=14.0
tempIon=12.3
sigma=3.03e+05
gamma=2.19e+05
LineShape=8.02e-07
norm=1.07e-15

A_10=6.24e-04

B_10=1.16e+04	B_01=3.48e+04

Collisional Rates
q_10: 4.13e-11	q_10*nHe: 2.06e+04

q_01: 2.72e-12	q_01*nHe: 1.36e+03

q_20: 3.93e-11	q_20*nHe: 1.96e+04

q_02: 2.09e-15	q_02*nHe: 1.04e+00

q_30: 1.33e-11	q_30*nHe: 6.65e+03

q_03: 1.06e-20	q_03*nHe: 5.29e-06

q_21: 1.25e-10	q_21*nHe: 6.23e+04

q_12: 1.00e-13	q_12*nHe: 5.02e+01

q_31: 8.49e-11	q_31*nHe: 4.25e+04

q_13: 1.03e-18	q_13*nHe: 5.13e-04

q_32: 6.81e-11	q_32*nHe: 3.41e+04

q_23: 1.02e-15	q_23*nHe: 5.10e-01



In [143]:
N = boltzman_distribution(Energy, 5.7)
N_He = [0, 0]
lightON=True
includeSpontaneousEmission = True
testMode = True
dN = computeRateDistributionEquations(1, [*N, 0, 0])
dN

counts=[0.9381286987741241, 0.06182146186192903, 4.9838617110216355e-05, 7.468368024180652e-10, 0, 0]

collisional_collection: i=0	collections=[2.2737367544323206e-13, 1.1102230246251565e-16, 0.0]
excitedLevel=1	B_rate=31897.315910849986	-B_rate=-31897.315910849986
einstein_collection: i=0	collections=[3.8576592201843713e-05, -31897.315910849986]
rateCollection=[[2.2737367544323206e-13, 1.1102230246251565e-16, 0.0, 3.8576592201843713e-05, -31897.315910849986]]
collisional_collection: i=1	collections=[-2.2737367544323206e-13, 4.440892098500626e-15, 4.0657581468206416e-20]
excitedLevel=1	B_rate=31897.315910849986	-B_rate=-31897.315910849986
einstein_collection: i=1	collections=[-3.8576592201843713e-05, 31897.315910849986]
rateCollection=[[2.2737367544323206e-13, 1.1102230246251565e-16, 0.0, 3.8576592201843713e-05, -31897.315910849986], [-2.2737367544323206e-13, 4.440892098500626e-15, 4.0657581468206416e-20, -3.8576592201843713e-05, 31897.315910849986]]
collisional_collection: i=2	collect

[-31897.315872273393,
 31897.315872273393,
 -4.551914400963142e-15,
 -4.0657581468206416e-20]

In [144]:
%%time

# nHe = float(rate_coefficients["He density(cm3)"])
# nHe = 1e15
# includeCollision = True
# includeAttachmentRate = False
# includeSpontaneousEmission = True

testMode = False
t = 600e-3 # in ms
tspan = [0, t]
N = boltzman_distribution(Energy, 300)
N_He = [0, 0]
boltzman_distribution_source = (N, [*N, 0, 0])[includeAttachmentRate]
print(f"{boltzman_distribution_source=}")

testMode = False
print(f"LightOFF")
lightON=False
N_rates_off = solve_ivp(computeRateDistributionEquations, tspan, boltzman_distribution_source, dense_output=True)

print(N_rates_off.sol(simulateTime).T[-1])
if not includeAttachmentRate:
    %matplotlib widget
    simulateTime = np.linspace(0, t, 100)
    simulateCounts_OFF = N_rates_off.sol(simulateTime)

    fig, ax = plt.subplots(figsize=(7, 4), dpi=100)
    legends = [f"N{i}" for i in range(totallevel)]

    ax.plot(simulateTime.T*1e3, simulateCounts_OFF.T)

    ax.plot(simulateTime*1e3, simulateCounts_OFF.sum(axis=0), "k")
    ax.legend(legends, title="OFF")

    ax.set(yscale="linear", xlabel="Time (ms)")

boltzman_distribution_source=array([0.08101779, 0.22604505, 0.32587477, 0.36706238])
LightOFF
[ 2.23099790e+21 -2.54751926e+23  2.61882922e+23 -9.36199393e+21]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wall time: 4.83 s


In [145]:
%%time
print(f"LightON")
lightON=True
N_rates_on = solve_ivp(computeRateDistributionEquations, tspan, boltzman_distribution_source, dense_output=True)

LightON
Wall time: 5.25 s


In [146]:
%matplotlib widget
simulateTime = np.linspace(0, t, 100)
simulateCounts_ON = N_rates_on.sol(simulateTime)
simulateCounts_OFF = N_rates_off.sol(simulateTime)

print(f"{simulateCounts_ON.shape=}\n{simulateCounts_OFF.shape=}")
fig, (ax, ax1) = plt.subplots(ncols=2, figsize=(12, 4), dpi=100)

legends = [f"N{i}" for i in range(totallevel)]
legends.append("NHe")
legends.append("NHe2")

simulateTime = simulateTime*1e3
counter = 0
for on, off in zip(simulateCounts_ON, simulateCounts_OFF):
    ax.plot(simulateTime, on, f"-C{counter}", label=legends[counter])
    ax.plot(simulateTime, off, f"--C{counter}")
    counter += 1
    
ax.plot(simulateTime, simulateCounts_ON.sum(axis=0), "k")

ax.legend(title=f"-ON, --OFF")
ax.set(yscale="log", ylabel="Counts", xlabel="Time(ms)")
ax.minorticks_on()

signal = (1 - (simulateCounts_ON[-2][1:] / simulateCounts_OFF[-2][1:]))*100
ax1.plot(simulateTime[1:], signal)
ax1.legend([f"Max. Signal = {signal.max():.2f} at {(simulateTime[1:][signal.argmax()]):.2f}ms"])
ax1.minorticks_on()
ax1.set(title="Signal as a function of trap time", xlabel="Time (ms)", ylabel="Signal (%)")
plt.tight_layout()

print(f"Signal: {(1 - (simulateCounts_ON[-2][-1] / simulateCounts_OFF[-2][-1]))*100:.2f}%")

simulateCounts_ON.shape=(4, 100)
simulateCounts_OFF.shape=(4, 100)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Signal: -562.89%


In [148]:
boltzman_distribution_source

array([0.08101779, 0.22604505, 0.32587477, 0.36706238])

In [149]:
boltzman_distribution(Energy, 5)

array([9.62813523e-01, 3.71761843e-02, 1.02927741e-05, 3.10689521e-11])